# Notebook for cleaning `CAREPLAN TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text

In [2]:
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")

In [3]:

def sql(query , engine = engine):
    return pd.read_sql(query,engine)

In [5]:
sql('select * from careplans limit 5')

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,225358003,Wound care,284551006.0,Laceration of foot


In [6]:
query = '''
    alter table careplans
    drop column code,
    drop column reasoncode;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [7]:
sql('select * from careplans limit 5')

,Id,START,STOP,PATIENT,ENCOUNTER,DESCRIPTION,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,Respiratory therapy,Acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,Respiratory therapy,Acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,Respiratory therapy,Acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,Respiratory therapy,Acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,Wound care,Laceration of foot


In [8]:
query = '''
    alter table careplans
    rename column REASONDESCRIPTION to REASON;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [9]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'careplans' and
    table_schema = database();
'''
sql(query)
careplans_columns = [c[0] for c in sql(query).values.tolist()]

In [10]:
careplans_columns

['DESCRIPTION', 'ENCOUNTER', 'Id', 'PATIENT', 'REASON', 'START', 'STOP']

In [11]:
for col in careplans_columns:
    query = f'''
        alter table careplans
        rename column {col} to {col.lower()}
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [12]:
sql('select * from careplans limit 5')

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,Respiratory therapy,Acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,Respiratory therapy,Acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,Respiratory therapy,Acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,Respiratory therapy,Acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,Wound care,Laceration of foot


In [13]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'careplans' and
    table_schema = database();
'''
sql(query)
careplans_columns = [c[0] for c in sql(query).values.tolist()]

In [15]:
for col in careplans_columns:
    query = f'''
        update careplans
        set {col} = lower({col})
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [16]:
sql('select * from careplans limit 5')

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,respiratory therapy,acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,respiratory therapy,acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,respiratory therapy,acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,respiratory therapy,acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,wound care,laceration of foot


In [18]:
sql('describe careplans')

,Field,Type,Null,Key,Default,Extra
0,id,text,YES,,None,
1,start,text,YES,,None,
2,stop,text,YES,,None,
3,patient,text,YES,,None,
4,encounter,text,YES,,None,
5,description,text,YES,,None,
6,reason,text,YES,,None,


In [19]:
query = '''
    alter table careplans
    modify start date,
    modify stop date;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [20]:
sql('describe careplans')

,Field,Type,Null,Key,Default,Extra
0,id,text,YES,,None,
1,start,date,YES,,None,
2,stop,date,YES,,None,
3,patient,text,YES,,None,
4,encounter,text,YES,,None,
5,description,text,YES,,None,
6,reason,text,YES,,None,


In [21]:
for col in careplans_columns:
    query = f'''
        select sum({col} is null) as {col}_null from careplans
        having {col}_null > 0;
    '''
    display(sql(query))

,description_null


,encounter_null


,id_null


,patient_null


,reason_null
0,327.0


,start_null


,stop_null
0,1532.0


In [24]:
query = '''
    update careplans
    set reason = 'others' where reason is null;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [25]:
sql('select * from careplans limit 5')

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,respiratory therapy,acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,respiratory therapy,acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,respiratory therapy,acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,respiratory therapy,acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,wound care,laceration of foot
